# 🛒 Smart Shopper Swarm

In [111]:
import sys, os
_jupyter_stdout = sys.stdout
_jupyter_stderr = sys.stderr
os.environ["TERM"] = "dumb"
os.environ["NO_COLOR"] = "1"

class SilentConsole:
    def __init__(self, *args, **kwargs): pass
    def print(self, *args, **kwargs): pass
    def log(self, *args, **kwargs): pass
    def __enter__(self): return self
    def __exit__(self, *args): pass

import types
if 'rich' not in sys.modules:
    r = types.ModuleType('rich')
    rc = types.ModuleType('rich.console')
    rc.Console = SilentConsole
    r.console = rc
    sys.modules['rich'] = r
    sys.modules['rich.console'] = rc

print("✅ OK")

In [112]:
from utils import get_openai_api_key
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"] = get_openai_api_key()
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY", "")
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool, ScrapeWebsiteTool
sys.stdout = _jupyter_stdout
sys.stderr = _jupyter_stderr
print("✅ OK")

In [119]:
# ✏️ MODIFIEZ ICI
produit = "iphone 15"
print(f"🔎 Produit: {produit}")

In [120]:
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

agent_chercheur = Agent(
    role="Chercheur de Prix",
    goal=f"Trouver AU MOINS 5 vendeurs pour '{produit}' avec prix et liens directs",
    backstory="Expert recherche. Trouve minimum 5 vendeurs avec liens directs.",
    tools=[search_tool, scrape_tool],
    verbose=False
)

agent_conseiller = Agent(
    role="Conseiller d'Achat",
    goal="Créer un rapport avec tableau comparatif et recommandation",
    backstory="Personal shopper. Rapport structuré avec tableaux Markdown.",
    verbose=False
)

print("✅ Agents OK")

In [121]:
tache_recherche = Task(
    description=f"Rechercher '{produit}' et trouver minimum 5 vendeurs avec prix et liens directs.",
    expected_output="Liste de vendeurs avec prix et liens.",
    agent=agent_chercheur
)

tache_rapport = Task(
    description=f"""Créer un rapport pour '{produit}'.
    
    FORMAT OBLIGATOIRE (utilise exactement ce format):
    
    ## 🏆 Recommandation
    **Vendeur:** [nom]
    **Prix:** [prix] EUR
    **Lien:** [url]
    
    ## 📊 Comparatif des Prix
    | Vendeur | Prix | Fiabilité | Lien |
    |---------|------|-----------|------|
    | [nom] | [prix] EUR | ⭐⭐⭐⭐⭐ | [url] |
    | [nom] | [prix] EUR | ⭐⭐⭐⭐ | [url] |
    (minimum 5 lignes)
    
    ## 💰 Économies
    - Prix min: [prix] EUR
    - Prix max: [prix] EUR  
    - Économie: [diff] EUR
    
    ## 💡 Conclusion
    [Explication courte]
    
    IMPORTANT: Utilise EUR au lieu de symboles monétaires.
    """,
    expected_output="Rapport structuré avec tableau Markdown.",
    agent=agent_conseiller
)

print("✅ Tâches OK")

In [122]:
crew = Crew(
    agents=[agent_chercheur, agent_conseiller],
    tasks=[tache_recherche, tache_rapport],
    process=Process.sequential,
    verbose=False
)
print("✅ Crew prêt")

In [123]:
print(f"🛒 Recherche: {produit}")
print("⏳ Patience...")
sys.stdout = _jupyter_stdout
sys.stderr = _jupyter_stderr
resultat = crew.kickoff()
print("\n✅ TERMINÉ")

In [124]:
from IPython.display import display, Markdown

if resultat:
    result_str = str(resultat.raw) if hasattr(resultat, 'raw') else str(resultat)
    # Remplacer $ par EUR pour éviter bug LaTeX
    result_str = result_str.replace('$', ' EUR').replace('€', ' EUR')
    display(Markdown(result_str))
else:
    print("Aucun résultat.")

## 🏆 Recommandation
**Vendeur:** Back Market  
**Prix:** 650 EUR  
**Lien:** https://www.backmarket.com/en-us/p/iphone-15?srsltid=AfmBOor9MPEnsp0TA6Z2C9FhvnNNYN1bMURvBpBO_iTMCiEULZt0rqv2

## 📊 Comparatif des Prix
| Vendeur                    | Prix       | Fiabilité    | Lien                                                                                                                  |
|---------------------------|------------|--------------|-----------------------------------------------------------------------------------------------------------------------|
| Back Market (Refurbished) | 650 EUR    | ⭐⭐⭐⭐         | https://www.backmarket.com/en-us/p/iphone-15?srsltid=AfmBOor9MPEnsp0TA6Z2C9FhvnNNYN1bMURvBpBO_iTMCiEULZt0rqv2      |
| Apple Refurbished Store   | 730 EUR*   | ⭐⭐⭐⭐⭐       | https://www.apple.com/shop/refurbished/iphone/iphone-15-pro                                                          |
| Amazon (Renewed)          | 720 EUR*   | ⭐⭐⭐⭐         | https://www.amazon.com/Apple-iPhone-15-128GB-Black/dp/B0CMPMY9ZZ                                                      |
| eBay (Very Good Used)     | 365 EUR**  | ⭐⭐⭐          | https://www.ebay.com/b/Apple-iPhone-15/9355/bn_7121790163                                                             |
| Swappa (Used)             | 600 EUR*   | ⭐⭐⭐⭐         | https://swappa.com/buy/apple-iphone-15?srsltid=AfmBOorFkqhD0qGRDeWIv8x0tQV_UvIQXqjIErensrfCR68kE7D85dVW              |
| Best Buy (New)            | 830 EUR*   | ⭐⭐⭐⭐⭐       | https://www.bestbuy.com/product/apple-iphone-15-128gb-unlocked-blue/JJGCQX68JS                                         |

\*Prix estimés basés sur la conversion USD/EUR et les fourchettes typiques.  
\**Prix estimé eBay converti de 379.43 USD.

## 💰 Économies
- Prix min: 365 EUR (eBay - Très bon état)  
- Prix max: 830 EUR (Best Buy - Neuf)  
- Économie: 465 EUR

## 💡 Conclusion
Pour un iPhone 15, le meilleur compromis qualité/prix se trouve chez Back Market avec un modèle reconditionné bien garanti à environ 650 EUR, offrant une économie significative par rapport au neuf tout en assurant une bonne fiabilité. Les options d'occasion très bon état sur eBay sont les moins chères mais avec plus de risques. Apple offre une sécurité maximale sur les modèles reconditionnés Pro, mais à un prix plus élevé. Pour un appareil neuf, Best Buy est l'option principale mais avec un coût nettement supérieur.